# Split a PDF
## Author: 2d Lt Marc Allerheiligen, USAFA DFVR
This program take a PDF of all Transcripts
and splits it into a separate PDF for each student.

## Library Installation

Library installation must be run once per machine this code is used on.  
There is no problem with running the whole file, but if the libraries are installed, then the program will run faster if you only run code after library installation.

In [ ]:
!pip install pypdf2
!pip install pycryptodome

# Parameters and Inputs

In [ ]:
REMOVE_FROM_NAME = [
    ]
OUT_FOLDER = "PDFs named correctly/"
default_doc_path = "in.pdf"

doc_path = input(
"""Make sure that the file with all Transcripts
is in the same folder as this program.
                 
What is the name of the file with all Transcripts?
If it is """+default_doc_path+" then just press enter.")
if len(doc_path) == 0:
    doc_path = default_doc_path

# Libraries

In [ ]:
from PyPDF2 import PdfWriter, PdfReader
from os import path, makedirs
from tqdm import tqdm

# Functions

In [ ]:
def flipName(nameIn):
    assert not ',' in nameIn, nameIn
    nameIn = nameIn.replace('.','')
    for badStr in REMOVE_FROM_NAME:
        nameIn = nameIn.replace(badStr, '')
    while '  ' in nameIn:
        nameIn = nameIn.replace('  ',' ')
    split = nameIn.strip().split()
    if len(split) == 1:
        return split[0]
    elif len(split) == 2:
        return split[-1]+', '+split[0]
    elif len(split) == 3:
        return split[-1]+', '+split[0]+' '+split[1]
    elif len(split) == 4:
        return split[-2]+' '+split[-1]+', '+split[0]+' '+split[1]
    else:
        return nameIn

# Main

In [ ]:
reader = PdfReader(open(doc_path, "rb"))

print("Reading names from pages...")
pages = dict()
numPages = len(reader.pages)
for pageNum in tqdm(range(numPages)):
    page = reader.pages[pageNum]
    text = page.extract_text()
    
    name_and_after = text.split("Name:")[1]
    name = name_and_after.split("\n")[0]
    name = name.strip()
    if name in pages.keys():
        pages[name].append(pageNum)
    else:
        pages[name] = [pageNum]
print("Names are read from all pages.")

# for every person, find their pages and export them
print("Saving files...")
if not path.exists(OUT_FOLDER):
    print("Creating folder...")
    makedirs(OUT_FOLDER)
for name in tqdm(pages.keys()):
    output = PdfWriter()
    for pageNum in pages[name]:
        output.add_page(reader.pages[pageNum])
    name = flipName(name)
    folderPath = OUT_FOLDER+name+'/'
    if not path.isdir(folderPath): makedirs(folderPath)
    fileName = name+' (T)'
    fileName = fileName + '.pdf'
    filePath = folderPath+fileName
    with open(filePath,'wb') as outputStream:
        output.write(outputStream)
print("Done.  Transcripts are in a folder called")
print(OUT_FOLDER)